In [1]:
import time
start_time = time.time()

In [2]:
import pandas as pd
data1 = pd.read_csv("VerizonFinancialsCleaned.csv")
data1.head()

,Date,StockPrice,TTMNetEPS,TTMSalesperShare,PricetoSalesRatio,BookValueperShare,TTMFCFperShare,QuarterlyRevenue,GrossProfit,OperatingIncome,...,CurrentLiabilities,CurrentAssetsInventory,QuickRatio,DebtToEquity Ratio,TTMNetIncome,LTInvestmentsDebt,TotalAssets,CashOnHand,LongTermDebt,TotalLiabilities
0,12/31/2006,19.46,2.12,29.96,0.65,16.67,6.23,22606,13454,3444,...,32.28,21.02,0.65,0.59,13.37,77.18,188804,5653,28646,140269
1,3/31/2007,20.04,2.07,30.56,0.66,16.80,5.66,22584,13672,3796,...,27.91,16.11,0.58,0.58,13.99,76.86,184284,3450,28073,135502
2,6/30/2007,21.98,2.10,31.14,0.71,17.17,5.41,23273,14042,4149,...,24.45,15.24,0.62,0.59,14.93,79.21,184760,2356,29374,134929
3,9/30/2007,23.88,1.88,31.66,0.75,17.19,4.96,23772,14164,4210,...,24.16,15.25,0.63,0.59,15.60,79.07,185619,1909,29383,135930
4,12/31/2007,23.78,1.90,32.18,0.74,17.58,5.78,23840,14044,3423,...,24.74,16.97,0.69,0.56,15.58,78.78,186959,3397,28203,136378


In [3]:
import numpy as np
# normalize data with natural log of difference between each day, add as extra row
# lognormal = percentage change per day, but can add or subtract cumulatively, unlike percentage
data1['ln StockPrice']=np.log(data1.StockPrice/data1.StockPrice.shift(1))
data1.head()

,Date,StockPrice,TTMNetEPS,TTMSalesperShare,PricetoSalesRatio,BookValueperShare,TTMFCFperShare,QuarterlyRevenue,GrossProfit,OperatingIncome,...,CurrentAssetsInventory,QuickRatio,DebtToEquity Ratio,TTMNetIncome,LTInvestmentsDebt,TotalAssets,CashOnHand,LongTermDebt,TotalLiabilities,ln StockPrice
0,12/31/2006,19.46,2.12,29.96,0.65,16.67,6.23,22606,13454,3444,...,21.02,0.65,0.59,13.37,77.18,188804,5653,28646,140269,NaN
1,3/31/2007,20.04,2.07,30.56,0.66,16.80,5.66,22584,13672,3796,...,16.11,0.58,0.58,13.99,76.86,184284,3450,28073,135502,0.029369
2,6/30/2007,21.98,2.10,31.14,0.71,17.17,5.41,23273,14042,4149,...,15.24,0.62,0.59,14.93,79.21,184760,2356,29374,134929,0.092403
3,9/30/2007,23.88,1.88,31.66,0.75,17.19,4.96,23772,14164,4210,...,15.25,0.63,0.59,15.60,79.07,185619,1909,29383,135930,0.082908
4,12/31/2007,23.78,1.90,32.18,0.74,17.58,5.78,23840,14044,3423,...,16.97,0.69,0.56,15.58,78.78,186959,3397,28203,136378,-0.004196


In [4]:
#create new dataframe copy the columns and normalize the StockPrice column
#lognormal = percentage change per day, but can add or subtract cumulatively, unlike percentage
data1norm = data1.filter(['Date'], axis=1)
data1norm['StockPrice'] = data1['StockPrice']
data1norm['Log Normal StockPrice']=np.log(data1.StockPrice/data1.StockPrice.shift(1))

In [5]:
data1norm.head()

,Date,StockPrice,Log Normal StockPrice
0,12/31/2006,19.46,NaN
1,3/31/2007,20.04,0.029369
2,6/30/2007,21.98,0.092403
3,9/30/2007,23.88,0.082908
4,12/31/2007,23.78,-0.004196


In [10]:
# check for all rows with null values eliminate at the end
nan = data1norm[data1norm.isnull().any(axis=1)]
nan

,Date,StockPrice,Log Normal StockPrice
0,12/31/2006,19.46,NaN


In [13]:
#create new dataframe for ratios
data1ratios = data1norm.filter(['Date'], axis=1)
data1ratios['ln StockPrice']=data1['ln StockPrice']

#calculate the ratios

#PE Ratio = StockPrice/TTMNetEPS
#Description: 
data1ratios['PE Ratio'] = np.around((data1['StockPrice']/data1['TTMNetEPS']), decimals=2)

#PricetoBook Ratio = StockPrice/BookValueperShare
#Description: 
data1ratios['PricetoBook Ratio'] = np.around((data1['StockPrice'])/data1['BookValueperShare'], decimals=2)

#Current Ratio = TotalAssets/TotalLiabilities
#Description: 
data1ratios['Current Ratio'] = np.around((data1['CurrentAssets']/data1['CurrentLiabilities']), decimals=2)

#Shareholder Equity = TotalAssets - TotalLiabilities
#Description: 
data1ratios['Shareholder Equity'] = (data1['TotalAssets'] - data1['TotalLiabilities'])

#DebtToEquity Ratio = LongTermDebt / Shareholder Equity
#Description: 
data1ratios['DebtToEquity Ratio'] = np.around((data1['LongTermDebt'] / data1ratios['Shareholder Equity']), decimals=2)

#ROI = TTMNetIncome/LTInvestmentsDebt
#Description: 
data1ratios['ROI'] = (data1['TTMNetIncome']/data1['LTInvestmentsDebt'])

data1ratios.head()

,Date,ln StockPrice,PE Ratio,PricetoBook Ratio,Current Ratio,Shareholder Equity,DebtToEquity Ratio,ROI
0,12/31/2006,NaN,9.18,1.17,0.70,48535,0.59,0.173231
1,3/31/2007,0.029369,9.68,1.19,0.64,48782,0.58,0.182019
2,6/30/2007,0.092403,10.47,1.28,0.70,49831,0.59,0.188486
3,9/30/2007,0.082908,12.70,1.39,0.71,49689,0.59,0.197294
4,12/31/2007,-0.004196,12.52,1.35,0.76,50581,0.56,0.197766


In [14]:
#Make data frame for Net Income and Revenue as percentages here
data1percentages = data1.filter(['Date'], axis=1)
data1percentages['Net Income % Change'] = (((data1.NetIncome - data1.NetIncome.shift(1))/data1.NetIncome.shift(1))*100)
data1percentages['Revenue % Change'] = (((data1.QuarterlyRevenue - data1.QuarterlyRevenue.shift(1))/data1.QuarterlyRevenue.shift(1))*100)
data1percentages.head()


,Date,Net Income % Change,Revenue % Change
0,12/31/2006,NaN,NaN
1,3/31/2007,44.864341,-0.097319
2,6/30/2007,12.575251,3.050832
3,9/30/2007,-24.480095,2.144115
4,12/31/2007,-15.656963,0.286051


In [31]:
#categorize current ratio numerical values
# Current Ratio
bins = [0, .5, 1.0, 1.5, 2.0, np.inf]
names = ['Very Low', 'Low', 'Acceptable', 'Good', 'Very Good']
data1ratios['Current Ratio Category'] = pd.cut(data1ratios['Current Ratio'], bins, labels=names)
# Price change categories
bins = [np.NINF,-.12, -.06, 0, .06, .12, np.inf]
names = [ 'Extremely Negative','Very Negative', 'Negative', 'Positive', 'Very Positive','Extremely Positive']
data1ratios['LN Price Change'] = pd.cut(data1ratios['ln StockPrice'], bins, labels=names)
data1ratios.head()

,Date,ln StockPrice,PE Ratio,PricetoBook Ratio,Current Ratio,Shareholder Equity,DebtToEquity Ratio,ROI,Current Ratio Category,Lognorm Price Category,LN Price Change
0,12/31/2006,NaN,9.18,1.17,0.70,48535,0.59,0.173231,Low,NaN,NaN
1,3/31/2007,0.029369,9.68,1.19,0.64,48782,0.58,0.182019,Low,Positive,Positive
2,6/30/2007,0.092403,10.47,1.28,0.70,49831,0.59,0.188486,Low,Very Positive,Very Positive
3,9/30/2007,0.082908,12.70,1.39,0.71,49689,0.59,0.197294,Low,Very Positive,Very Positive
4,12/31/2007,-0.004196,12.52,1.35,0.76,50581,0.56,0.197766,Low,Negative,Negative


In [32]:
# combine all preproccessed data to a final data frame
data1final = data1.filter(['Date'], axis=1)
data1final['StockPrice'] = data1['StockPrice']
data1final['Log Normal StockPrice'] = data1norm['Log Normal StockPrice']
data1final['LN Price Change'] = data1ratios['LN Price Change']
data1final['TTMNetEPS'] = data1['TTMNetEPS']
data1final['PE Ratio'] = data1ratios['PE Ratio']
data1final['TTMSalesperShare'] = data1['TTMSalesperShare']
data1final['PricetoSalesRatio'] = data1['PricetoSalesRatio']
data1final['BookValueperShare'] = data1['BookValueperShare']
data1final['PricetoBook Ratio'] = data1ratios['PricetoBook Ratio']
data1final['TTMFCFperShare'] = data1['TTMFCFperShare']
data1final['QuarterlyRevenue'] = data1['QuarterlyRevenue']
data1final['Quarterly Revenue % Change'] = np.around(data1percentages['Revenue % Change'], decimals=2)
data1final['GrossProfit'] = data1['GrossProfit']
data1final['OperatingIncome'] = data1['OperatingIncome']
data1final['EBITDA'] = data1['EBITDA']
data1final['NetIncome'] = data1['NetIncome']
data1final['Net Income % Change'] = np.around(data1percentages['Net Income % Change'], decimals=2)
data1final['EPS'] = data1['EPS']
data1final['CurrentAssets'] = data1['CurrentAssets']
data1final['CurrentLiabilities'] = data1['CurrentLiabilities']
data1final['CurrentAssetsInventory'] = data1['CurrentAssetsInventory']
data1final['OperatingIncome'] = data1['CurrentAssetsInventory']
data1final['QuickRatio'] = data1['QuickRatio']
data1final['DebtToEquity Ratio'] = data1['DebtToEquity Ratio']
data1final['TTMNetIncome'] = data1['TTMNetIncome']
data1final['LTInvestmentsDebt'] = data1['LTInvestmentsDebt']
data1final['ROI'] = data1ratios['ROI']
data1final['TotalAssets'] = data1['TotalAssets']
data1final['TotalLiabilities'] = data1['TotalLiabilities']
data1final['Current Ratio'] = data1ratios['Current Ratio']
data1final['Current Ratio Category'] = data1ratios['Current Ratio Category']
data1final['CashOnHand'] = data1['CashOnHand']
data1final['LongTermDebt'] = data1['LongTermDebt']
data1final['Shareholder Equity'] = data1ratios['Shareholder Equity']
data1final['DebtToEquity Ratio'] = data1ratios['DebtToEquity Ratio']

In [33]:
# check for all rows with null values
nan = data1final[data1final.isnull().any(axis=1)]
nan

,Date,StockPrice,Log Normal StockPrice,LN Price Change,TTMNetEPS,PE Ratio,TTMSalesperShare,PricetoSalesRatio,BookValueperShare,PricetoBook Ratio,...,TTMNetIncome,LTInvestmentsDebt,ROI,TotalAssets,TotalLiabilities,Current Ratio,Current Ratio Category,CashOnHand,LongTermDebt,Shareholder Equity
0,12/31/2006,19.46,NaN,NaN,2.12,9.18,29.96,0.65,16.67,1.17,...,13.37,77.18,0.173231,188804,140269,0.7,Low,5653,28646,48535


In [36]:
#drop the first row which contains null value
#To improve make this code work to drop any rows with null values instead of manually seeing its just the first row
data1final = data1final.drop(data1final.index[0])
data1final.head()

,Date,StockPrice,Log Normal StockPrice,LN Price Change,TTMNetEPS,PE Ratio,TTMSalesperShare,PricetoSalesRatio,BookValueperShare,PricetoBook Ratio,...,TTMNetIncome,LTInvestmentsDebt,ROI,TotalAssets,TotalLiabilities,Current Ratio,Current Ratio Category,CashOnHand,LongTermDebt,Shareholder Equity
2,6/30/2007,21.98,0.092403,Very Positive,2.10,10.47,31.14,0.71,17.17,1.28,...,14.93,79.21,0.188486,184760,134929,0.70,Low,2356,29374,49831
3,9/30/2007,23.88,0.082908,Very Positive,1.88,12.70,31.66,0.75,17.19,1.39,...,15.60,79.07,0.197294,185619,135930,0.71,Low,1909,29383,49689
4,12/31/2007,23.78,-0.004196,Negative,1.90,12.52,32.18,0.74,17.58,1.35,...,15.58,78.78,0.197766,186959,136378,0.76,Low,3397,28203,50581
5,3/31/2008,20.04,-0.171115,Extremely Negative,1.96,10.22,32.74,0.61,17.64,1.14,...,16.12,82.41,0.195607,191293,141013,0.91,Low,7442,32134,50280
6,6/30/2008,19.78,-0.013059,Negative,2.04,9.70,33.19,0.60,17.87,1.11,...,16.51,84.67,0.194992,200646,149759,0.60,Low,2074,33778,50887


In [37]:
#highlight new columns
def highlight_cols(s):
    color = 'yellow'
    return 'background-color: %s' % color

data1final.style.applymap(highlight_cols, subset=pd.IndexSlice[:, ['Log Normal StockPrice', 'PE Ratio', 'PricetoBook Ratio',
                        'Quarterly Revenue % Change','Net Income % Change','Current Ratio', 'Shareholder Equity', 'ROI', 'Current Ratio Category']])


,Date,StockPrice,Log Normal StockPrice,LN Price Change,TTMNetEPS,PE Ratio,TTMSalesperShare,PricetoSalesRatio,BookValueperShare,PricetoBook Ratio,TTMFCFperShare,QuarterlyRevenue,Quarterly Revenue % Change,GrossProfit,OperatingIncome,EBITDA,NetIncome,Net Income % Change,EPS,CurrentAssets,CurrentLiabilities,CurrentAssetsInventory,QuickRatio,DebtToEquity Ratio,TTMNetIncome,LTInvestmentsDebt,ROI,TotalAssets,TotalLiabilities,Current Ratio,Current Ratio Category,CashOnHand,LongTermDebt,Shareholder Equity
2,6/30/2007,21.98,0.0924027,Very Positive,2.1,10.47,31.14,0.71,17.17,1.28,5.41,23273,3.05,14042,15.24,7722,1683,12.58,0.58,17.03,24.45,15.24,0.62,0.59,14.93,79.21,0.188486,184760,134929,0.7,Low,2356,29374,49831
3,9/30/2007,23.88,0.0829083,Very Positive,1.88,12.7,31.66,0.75,17.19,1.39,4.96,23772,2.14,14164,15.25,7815,1271,-24.48,0.44,17.04,24.16,15.25,0.63,0.59,15.6,79.07,0.197294,185619,135930,0.71,Low,1909,29383,49689
4,12/31/2007,23.78,-0.0041964,Negative,1.9,12.52,32.18,0.74,17.58,1.35,5.78,23840,0.29,14044,16.97,7089,1072,-15.66,0.37,18.7,24.74,16.97,0.69,0.56,15.58,78.78,0.197766,186959,136378,0.76,Low,3397,28203,50581
5,3/31/2008,20.04,-0.171115,Extremely Negative,1.96,10.22,32.74,0.61,17.64,1.14,6.07,23833,-0.03,14316,20.78,7915,3049,184.42,0.57,22.45,24.74,20.78,0.84,0.64,16.12,82.41,0.195607,191293,141013,0.91,Low,7442,32134,50280
6,6/30/2008,19.78,-0.013059,Negative,2.04,9.7,33.19,0.6,17.87,1.11,6.71,24124,1.22,14658,15.78,8130,3404,11.64,0.66,18,29.9,15.78,0.53,0.66,16.51,84.67,0.194992,200646,149759,0.6,Low,2074,33778,50887
7,9/30/2008,18.15,-0.0860008,Very Negative,2.19,8.29,33.7,0.54,17.94,1.01,7.69,24752,2.6,14704,16.65,7825,1669,-50.97,0.59,19.13,28.28,16.65,0.59,0.74,16.48,88.43,0.186362,204861,153907,0.68,Low,1674,37478,50954
8,12/31/2008,19.49,0.071231,Very Positive,2.25,8.66,34.13,0.57,27.78,0.7,8.1,24645,-0.43,15061,23.98,-6648,-10315,-718.03,-2.59,26.08,25.91,23.98,0.93,0.6,2.61,125.86,0.0207373,202352,123447,1.01,Acceptable,10291,46959,78905
9,3/31/2009,17.62,-0.100867,Very Negative,2.26,7.8,35.17,0.5,28.36,0.62,8.72,26591,7.9,16283,19.68,8722,1645,-115.95,0.58,21.88,35.07,19.68,0.56,0.69,2.97,136.23,0.0218014,226851,146297,0.62,Low,4351,55674,80554
10,6/30/2009,18.19,0.0318374,Positive,2.12,8.58,36.17,0.5,29.16,0.62,9.36,26861,1.02,16380,18.69,8509,1483,-9.85,0.52,21.46,26.37,18.69,0.71,0.72,2.85,142.29,0.0200295,224999,142178,0.81,Low,1180,59469,82821
11,9/30/2009,18.2,0.000549602,Positive,1.94,9.38,37.06,0.49,29.78,0.61,10.53,27265,1.5,16269,19.47,8158,1176,-20.7,0.41,22.02,27.31,19.47,0.71,0.68,2.66,141.96,0.0187377,226404,141822,0.81,Low,1690,57374,84582


In [38]:
#export to csv
data1final.to_csv('VerizonPreProcessed.csv')
print("Preprocessing Completed In: %s seconds" % (time.time() - start_time))

Preprocessing Completed In: 4918.390425205231 seconds
